# Walk forward validation

In [1]:

from modules import utils
utils.configure_plotly_template(showlegend=True)

## Load data

In [ ]:
import pandas as pd

df = pd.read_parquet('../../../data/UCIrvine/ElectricityLoadDiagrams20112014.parquet').asfreq('h').div(1_000)
df.columns = ['values']

df

In [3]:
series = df['values'].copy()

## Data stationarity

#### Constant variance

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller(df["values"], maxlag=24)

In [ ]:
import numpy as np

df["values_log"] = np.log(df["values"])
df = df.dropna()
df

In [ ]:
adfuller(df["values_log"])

In [ ]:
fig = df.plot(facet_col="variable")
fig.update_yaxes(matches=None)
fig.update_layout()

In [ ]:
PERIODS = 24

utils.plot_decomposition_comparison(df["values_log"], period=PERIODS)

#### Constant mean

In [ ]:
df['values_log_diff'] = df['values_log'].diff().dropna()
df

In [ ]:
df = df.dropna()
df

In [ ]:
fig = df.plot(facet_col="variable")
fig.update_yaxes(matches=None)
fig.update_layout()

In [ ]:
adfuller(df["values_log_diff"], maxlag=24)

## ACF & PACF


In [13]:
COLUMN_VALUES = "values"

In [ ]:
x = df[COLUMN_VALUES]
x

In [15]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

In [ ]:
LAGS = 2.1 * PERIODS

fig, axes = plt.subplots(2, 1, figsize=(10, 6))

plot_acf(x, ax=axes[0], lags=LAGS)
axes[0].set_title("ACF of log-differenced series")

plot_pacf(x, ax=axes[1], lags=LAGS)
axes[1].set_title("PACF of log-differenced series")

plt.tight_layout()
plt.show()

## Model comparison: SARIMA vs ETS vs Prophet

In [17]:
from sklearn.model_selection import TimeSeriesSplit

### Experiment configuration

In [18]:
horizon = 24
tsv = TimeSeriesSplit(test_size=horizon, max_train_size=horizon*365*3)

In [ ]:
configs = {
    'sarima': {
        'model_params': {
            'order': (1, 0, 1),
            'seasonal_order': (1, 1, 0, 24),
            'enforce_stationarity': False,
            'enforce_invertibility': False,
        },
        'log_transform': False,
    },
    'ets': {
        'model_params': {
            'trend': 'add',
            'seasonal': 'add',
            'seasonal_periods': 24,
        },
        'log_transform': False,
    },
    'prophet': {
        'model_params': {
            'seasonality_mode': 'additive',
            'daily_seasonality': True,
        },
        'log_transform': False,
    },
}

configs

In [20]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

metrics = {
    'rmse': root_mean_squared_error,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error
}

### Run experiment with all models

In [ ]:
p, d, q = order = (1, 0, 1)
P, D, Q, s = seasonal_order = (1, 1, 0, 24)

idx_offset = max(p + d + q, s * (P + D + Q))  # conservative offset
idx_offset

In [ ]:
from sklearn.metrics import root_mean_squared_error
from modules.utils import TimeSeriesForecaster

d_metrics = []
d_forecasts = []

x = series.copy()
for fold, (train_idx, test_idx) in enumerate(tsv.split(series)):
    
    print(f"Fold {fold + 1}")
    
    x = x.tz_localize(None)
    
    data = {
        'train': x.iloc[train_idx],
        'test': x.iloc[test_idx],
    }
    
    tf = TimeSeriesForecaster(train=data['train'], test=data['test'], freq="h", idx_offset=idx_offset)
    
    df_forecast = tf.bulk_forecast(configs, metrics=metrics)
    df_forecast['fold'] = fold
    d_metrics.append(df_forecast)
    
    df_forecast = tf.combine_with_historical(df_forecast=df_forecast)
    df_forecast['fold'] = fold
    
    d_forecasts.append(df_forecast)

In [ ]:
d_metrics[0]

In [ ]:
d_forecasts[0]

### Visualize results

#### Forecasts plot

In [ ]:
df = pd.concat(d_forecasts)
df

In [ ]:
import plotly.express as px

fig = px.line(
    df,
    x="datetime",
    y="values",
    color="model",
    facet_col="fold",
    facet_row="split",
    category_orders={"split": ["train", "test"]},
    height=600,
    width=1200,
)

fig.update_yaxes(matches=None)
fig.update_xaxes(matches=None, tickangle=45)

#### Metrics

In [ ]:
dfs = []
for df in d_metrics:
    for i, x in df.iterrows():
        start, end = x.datetime[[0, -1]]
        dfs.append({
            'fold': x.fold,
            'split': x.split,
            'start': start,
            'end': end,
            'model': x.model,
            'rmse': x.rmse,
        })

df = pd.DataFrame(dfs)
df

In [ ]:
x = df.set_index(['fold', 'split', 'start', 'end', 'model']).unstack(level='model')
x.style.background_gradient(cmap='Greens_r', axis=None).format(precision=2)

In [ ]:
x.loc[:, 'test', :]['rmse'].mean()

## How `TimeSeriesSplit` works

In [ ]:
N_SPLITS = 5
TEST_SIZE = PERIODS * 1
TOTAL_SIZE = len(series)

MAX_TRAIN_SIZE = TOTAL_SIZE - TEST_SIZE * N_SPLITS
MAX_TRAIN_SIZE

### Configure the instance

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
ts = TimeSeriesSplit(n_splits=N_SPLITS, test_size=TEST_SIZE, max_train_size=MAX_TRAIN_SIZE)

ts

### Generate splits

In [ ]:
splits = ts.split(X=series)
splits

In [83]:
train, test = next(splits)

In [ ]:
train

In [ ]:
test

In [ ]:
len(test)

In [ ]:
series.iloc[train]

In [ ]:
series.iloc[test]

In [ ]:
series.iloc[train]

In [ ]:
series.iloc[test]

### Iterate over splits to train and test models

In [37]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
from sklearn.metrics import root_mean_squared_error

x = []
for fold, (train_idx, test_idx) in enumerate(tsv.split(series)):

    train = series.iloc[train_idx]
    test = series.iloc[test_idx]

    model = ExponentialSmoothing(
        train,
        trend="add",
        seasonal="mul",
        seasonal_periods=24,
    ).fit()
    
    y_pred = model.forecast(len(test))
    
    rmse = root_mean_squared_error(test, y_pred)
    x.append({
        'fold': fold,
        'train_start': train.index[0],
        'train_end': train.index[-1],
        'test_start': test.index[0],
        'test_end': test.index[-1],
        'rmse': rmse,
    })

In [ ]:
x = pd.DataFrame(x)
x.style

In [ ]:
x.rmse.mean()